In [3]:
import pandas as pd
import backtesting as bt
import pandas_ta as ta
import numpy as np
import datetime as dt
from IPython.display import clear_output
import breeze_client as bc

In [4]:
filename = 'data/NESIND/NESIND_5minute_90days.csv'
data = bc.read_csv(filename)
data

,Close,exchange_code,High,Low,Open,stock_code,Volume
datetime,,,,,,,
2024-01-29 09:05:00,2482.15,NSE,2482.15,2482.15,2482.15,NESIND,0
2024-01-29 09:15:00,2482.00,NSE,2488.90,2476.60,2482.15,NESIND,39718
2024-01-29 09:20:00,2497.00,NSE,2497.55,2481.90,2482.00,NESIND,22338
2024-01-29 09:25:00,2500.75,NSE,2505.00,2496.50,2496.95,NESIND,16544
2024-01-29 09:30:00,2499.25,NSE,2502.20,2499.25,2500.25,NESIND,8660
...,...,...,...,...,...,...,...
2024-04-26 15:05:00,2479.50,NSE,2479.95,2473.00,2477.40,NESIND,37317
2024-04-26 15:10:00,2481.60,NSE,2482.80,2479.55,2479.55,NESIND,30752
2024-04-26 15:15:00,2482.90,NSE,2485.70,2479.65,2481.65,NESIND,39890


In [5]:
def BBands_indicator(data,length,sd):
  bbands= ta.bbands(data.Close.s,length,sd)
  return bbands.to_numpy().T[:3]

def BBands_width(data,length,sd):
  bbands = ta.bbands(data.Close.s,length,sd)
  return bbands.to_numpy().T[3]

def BBands_per(data,length,sd):
  bbands = ta.bbands(data.Close.s,length,sd)
  return bbands.to_numpy().T[4]

In [10]:
class rsi(bt.Strategy):

    target = 9
    rsi_length = 14
    rsi_upperband = 70
    rsi_lowerband = 30
    bb_length = 20
    bb_sd = 2
    



    def init(self):
        self.rsi = self.I(ta.rsi,self.data.Close.s,self.rsi_length)
        self.vema = self.I(ta.ema,self.data.Volume.s,7)
        self.bbands = self.I(BBands_indicator,self.data,self.bb_length,self.bb_sd)

    
    def next(self):
        
        price = self.data.Close[-1]
        current_rsi = self.rsi[-1]

        vema = self.vema[-1]
        volume = self.data.Volume[-1]
        


        target= self.target/10
        target_precent = target/100
        target_price = (1-target_precent)*price


        

        if self.rsi_upperband <= current_rsi and vema < volume:
            if not self.position.is_short:
                self.sell(size = 200,tp=target_price,sl=1.001*price)
        
    

In [11]:
eng = bt.Backtest(data,rsi,cash=4000000,commission=0.002)

In [12]:
stats = eng.run()
stats

Start                     2024-01-29 09:05:00
End                       2024-04-26 15:25:00
Duration                     88 days 06:20:00
Exposure Time [%]                    8.429536
Equity Final [$]                   3932598.13
Equity Peak [$]                    4001888.03
Return [%]                          -1.685047
Buy & Hold Return [%]                0.517696
Return (Ann.) [%]                   -6.779724
Volatility (Ann.) [%]                0.983961
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -1.823549
Avg. Drawdown [%]                   -0.646578
Max. Drawdown Duration       84 days 00:05:00
Avg. Drawdown Duration       28 days 16:09:00
# Trades                                   65
Win Rate [%]                        10.769231
Best Trade [%]                       0.733618
Worst Trade [%]                     -0.887077
Avg. Trade [%]                    

In [13]:
eng.plot()

c:\Users\himan\AppData\Local\Programs\Python\Python310\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\Users\himan\AppData\Local\Programs\Python\Python310\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\Users\himan\AppData\Local\Programs\Python\Python310\lib\site-packages\backtesting\_plotting.py:455: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df2 = (df.assign(_width=1).set_index('datetime')
c:\Users\himan\AppData\Local\Programs\Python\Python310\lib\site-packages\backtesting\_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig =

GridPlot(id='p1424', ...)

In [9]:
def optim_func(series):

    if series['Win Rate [%]'] > 65 and series['# Trades'] > 30:
        return series['Return [%]']
    
    


    else:
        return -1000000

opt_stats,heatmap = eng.optimize(
    rsi_length = range(5,22),
    rsi_upperband = range(45,85),
    
    maximize = optim_func,
    
    return_heatmap = True,
    max_tries = 1000

)
opt_stats

c:\Users\himan\AppData\Local\Programs\Python\Python310\lib\site-packages\backtesting\backtesting.py:1488: UserWarning: Searching for best of 680 configurations.
  output = _optimize_grid()
c:\Users\himan\AppData\Local\Programs\Python\Python310\lib\site-packages\backtesting\_stats.py:122: RuntimeWarning: divide by zero encountered in scalar divide
  s.loc['Sortino Ratio'] = np.clip((annualized_return - risk_free_rate) / (np.sqrt(np.mean(day_returns.clip(-np.inf, 0)**2)) * np.sqrt(annual_trading_days)), 0, np.inf)  # noqa: E501
c:\Users\himan\AppData\Local\Programs\Python\Python310\lib\site-packages\backtesting\_stats.py:122: RuntimeWarning: divide by zero encountered in scalar divide
  s.loc['Sortino Ratio'] = np.clip((annualized_return - risk_free_rate) / (np.sqrt(np.mean(day_returns.clip(-np.inf, 0)**2)) * np.sqrt(annual_trading_days)), 0, np.inf)  # noqa: E501
c:\Users\himan\AppData\Local\Programs\Python\Python310\lib\site-packages\backtesting\_stats.py:122: RuntimeWarning: divide by

Start                     2024-01-29 09:05:00
End                       2024-04-26 15:25:00
Duration                     88 days 06:20:00
Exposure Time [%]                   83.380527
Equity Final [$]                   3165561.04
Equity Peak [$]                     4000000.0
Return [%]                         -20.860974
Buy & Hold Return [%]                0.517696
Return (Ann.) [%]                  -61.658674
Volatility (Ann.) [%]                1.702935
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -20.860974
Avg. Drawdown [%]                  -20.860974
Max. Drawdown Duration       88 days 05:30:00
Avg. Drawdown Duration       88 days 05:30:00
# Trades                                  838
Win Rate [%]                         15.75179
Best Trade [%]                       1.439869
Worst Trade [%]                     -1.020333
Avg. Trade [%]                    

In [61]:
opt_stats

Start                     2024-01-29 09:05:00
End                       2024-04-26 15:25:00
Duration                     88 days 06:20:00
Exposure Time [%]                   27.641037
Equity Final [$]                   4016567.24
Equity Peak [$]                    4028913.36
Return [%]                           0.414181
Buy & Hold Return [%]                0.517696
Return (Ann.) [%]                     1.72217
Volatility (Ann.) [%]                   1.465
Sharpe Ratio                         1.175543
Sortino Ratio                        1.844217
Calmar Ratio                          3.83626
Max. Drawdown [%]                   -0.448919
Avg. Drawdown [%]                   -0.080914
Max. Drawdown Duration       38 days 01:25:00
Avg. Drawdown Duration        2 days 00:38:00
# Trades                                   19
Win Rate [%]                        57.894737
Best Trade [%]                        1.47955
Worst Trade [%]                     -1.389335
Avg. Trade [%]                    

In [62]:
eng.plot()

C:\Users\Ankit\AppData\Roaming\Python\Python310\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
C:\Users\Ankit\AppData\Roaming\Python\Python310\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
C:\Users\Ankit\AppData\Roaming\Python\Python310\site-packages\backtesting\_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
C:\Users\Ankit\AppData\Roaming\Python\Python310\site-packages\backtesting\_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


GridPlot(id='p2463', ...)